In [67]:
import mysql.connector as connector
from mysql.connector import errorcode

# Establish connection between Python and MySQL database via connector API


try:
    connection=connector.connect(
                                 user = "", # your username
                                 password = "", # your password
                                 db = "" # your db
                                )
    print("Connection between MySQL and Python is established.\n")

except connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("connection user or password are incorrect")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("database does not exist")
    else:
        print(err)

# Create cursor object to communicate with entire MySQL database
cursor = connection.cursor(buffered=True)
print("Cursor is created to communicate with the MySQL using Python.\n")

Connection between MySQL and Python is established.

Cursor is created to communicate with the MySQL using Python.



In [52]:
#import MySQLCOnnectionPool, Error and connector
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import mysql.connector as connector
print("MySQLConnectionPool imported, Error imported, connector imported")

dbconfig={"database":"little_lemon", "user":"root", "password":"f?ZbmAdv!7.SJr"}

try:
    pool = MySQLConnectionPool(pool_name="pool_b",
    pool_size = 4,
    **dbconfig)

    print("The connection pool is created with a name:", pool.pool_name)
    print("The pool size is: ", pool.pool_size)

except Error as er:
    print("Error code: ", er.errno)
    print("Error Message: ", er.msg)

MySQLConnectionPool imported, Error imported, connector imported
The connection pool is created with a name: pool_b
The pool size is:  4


In [53]:
# Setting little_lemon for use 
cursor.execute("use little_lemon")

# Confirming
print("Database is use is:", connection.database)
print("The existing tables in the little_lemon database are:")
cursor.execute("SHOW TABLES") 
# Print table names 
for table in cursor: 
    print(table) 

Database is use is: little_lemon
The existing tables in the little_lemon database are:
('address',)
('bookings',)
('cities',)
('countries',)
('coursenames',)
('courses',)
('cuisines',)
('customers',)
('desserts',)
('drinks',)
('menu',)
('orders',)
('orders_view',)
('ordersitem',)
('sides',)
('starters',)
('timestamps',)


In [ ]:
# CourseNames view.
CourseNames_view = """
        CREATE VIEW courseNames AS SELECT m.MenuID, c.CourseName from little_lemon.courses c
        INNER JOIN little_lemon.menu AS m
        ON m.CourseID = c.CourseID ;"""
cursor.execute(CourseNames_view)




In [ ]:
# CourseNames view.
CourseNames_view = """
        CREATE VIEW orders_view AS SELECT o.OrderID, c.CustomerName, o.OrderDate, ct.City, co.CountryCode, co.Country, 
        COUNT(oi.OrdersItemID) AS item_count
        from little_lemon.orders o

        INNER JOIN little_lemon.address AS a
        ON o.AddressID = a.AddressID

        INNER JOIN little_lemon.ordersitem oi
        ON oi.OrderID=o.OrderID

        INNER JOIN little_lemon.cities AS ct
        ON a.CityID = ct.CityID

        INNER JOIN little_lemon.countries AS co
        ON ct.CountryID=co.CountryID

        INNER JOIN little_lemon.customers c
        ON o.CustomerID= c.CustomerID
        GROUP BY o.OrderID;

        SELECT * FROM little_lemon.orders_view;"""
cursor.execute(CourseNames_view)



In [ ]:
# add_createdate trigger.
add_createdate_trigger = """
        CREATE TRIGGER add_createdate BEFORE INSERT
        ON little_lemon.bookings
        FOR EACH ROW
        BEGIN
        SET NEW.CreatedAt = IFNULL(NEW.CreatedAt, NOW());
        END;        
"""
cursor.execute(add_createdate_trigger)


 

In [54]:
insert_bookings="""
INSERT IGNORE INTO Bookings (BookingID, BookingSlot, BookingDate, 
TableNo, GuestFirstName, GuestLastName, IsActive, CityID)
VALUES
(1, '19:00:00', '2025-01-01', 42, 'Batuhan', 'Baykal', 1, 3),
(2, '20:00:00', '2025-01-01', 100, 'Ayde', 'Baykal', 1, 2);"""

In [60]:
print("Inserting data in Bookings table.")
# Populate Bookings table
cursor.execute(insert_bookings)
print("Total number of rows in Bookings table: ", cursor.rowcount)
connection.commit()

Inserting data in Bookings table.
Total number of rows in Bookings table:  0


In [ ]:
all_bookings = """
        SELECT * FROM Bookings;"""
cursor.execute(all_bookings)

# Fetch all rows
result = cursor.fetchall()

# Print the result
for row in result:
    print(row)

(1, datetime.timedelta(seconds=68400), datetime.date(2025, 1, 1), 42, 'Batuhan', 'Baykal', 1, 3)
(2, datetime.timedelta(seconds=72000), datetime.date(2025, 1, 1), 100, 'Ayde', 'Baykal', 1, 2)
(7, datetime.timedelta(seconds=18000), datetime.date(2023, 1, 6), 1, 'Baran', 'Baykal', 1, 4)


In [75]:
# Drop the existing AddBooking procedure if it exists
drop_procedure_query = "DROP PROCEDURE IF EXISTS AddBooking"
cursor.execute(drop_procedure_query)

# Define the AddBooking stored procedure
add_booking_query = """
CREATE PROCEDURE AddBooking(
    IN p_BookingSlot TIME, 
    IN p_BookingDate DATE, 
    IN p_TableNo SMALLINT(100), 
    IN p_GuestFirstName VARCHAR(45), 
    IN p_GuestLastName VARCHAR(45),
    In p_CityID INTEGER
)
BEGIN
    -- Insert new booking
    INSERT INTO Bookings (BookingSlot, BookingDate, TableNo, GuestFirstName, GuestLastName, CityID)
    VALUES (p_BookingSlot, p_BookingDate, p_TableNo, p_GuestFirstName, p_GuestLastName, p_CityID);
END;
"""

# Create the updated AddBooking procedure
cursor.execute(add_booking_query)

In [77]:
# Call the AddBooking stored procedure with specific values
cursor.callproc("AddBooking", ['12:00:00', '2023-01-06', 4, 'ali', 'veli', 5])

('12:00:00', '2023-01-06', 4, 'ali', 'veli', 5)

In [ ]:
# Drop the existing UpdateBooking procedure if it exists
drop_procedure_query = "DROP PROCEDURE IF EXISTS UpdateBooking"
cursor.execute(drop_procedure_query)


# Create the UpdateBooking procedure
update_booking_query = """
CREATE PROCEDURE UpdateBooking(
    IN p_BookingID INT,  -- Unique identifier for the booking to be updated
    IN p_NewBookingSlot TIME, 
    IN p_NewBookingDate DATE, 
    IN p_NewTableNo SMALLINT(100), 
    IN p_NewGuestFirstName VARCHAR(45), 
    IN p_NewGuestLastName VARCHAR(45),
    IN p_IsActive TINYINT,
    IN p_CityID INTEGER
)
BEGIN
    -- Check if the booking with the given ID exists
    IF EXISTS (SELECT * FROM Bookings WHERE BookingID = p_BookingID) THEN
        -- Update the existing booking
        UPDATE Bookings
        SET
            BookingSlot = p_NewBookingSlot,
            BookingDate = p_NewBookingDate,
            TableNo = p_NewTableNo,
            GuestFirstName = p_NewGuestFirstName,
            GuestLastName = p_NewGuestLastName,
            IsActive = p_IsActive,
            CityID = p_CityID
        WHERE BookingID = p_BookingID;
        
        SELECT 'Booking updated successfully.' AS Message;
    ELSE
        SELECT 'Booking not found.' AS Message;
    END IF;
END;
"""

cursor.execute(update_booking_query)

In [ ]:
# Check if the procedure exists
cursor.execute("SHOW PROCEDURE STATUS WHERE Db = 'little_lemon' AND Name = 'UpdateBooking'")
result = cursor.fetchall()

if result:
    print("The stored procedure exists.")
else:
    print("The stored procedure does not exist.")

The stored procedure exists.


In [ ]:
# Execute the stored procedure
cursor.callproc("UpdateBooking", [3, '12:00:00', '2023-02-02', 1, 'Serdar', 'Baykal', 1, 4])
                                  

(3, '12:00:00', '2023-02-02', 1, 'Serdar', 'Baykal', 1, 4)

In [ ]:
all_bookings = """
        SELECT * FROM Bookings;"""
cursor.execute(all_bookings)

# Fetch all rows
result = cursor.fetchall()

# Print the result
for row in result:
    print(row)

(1, datetime.timedelta(seconds=68400), datetime.date(2025, 1, 1), 42, 'Batuhan', 'Baykal', 1, 3)
(2, datetime.timedelta(seconds=72000), datetime.date(2025, 1, 1), 100, 'Ayde', 'Baykal', 1, 2)
(7, datetime.timedelta(seconds=18000), datetime.date(2023, 1, 6), 1, 'Baran', 'Baykal', 1, 4)


In [ ]:
# Drop the existing CancelBooking procedure if it exists
drop_procedure_query = "DROP PROCEDURE IF EXISTS CancelBooking"
cursor.execute(drop_procedure_query)
connection.commit()

# Define the AddBooking stored procedure
cancel_booking_query = """
CREATE PROCEDURE CancelBooking(
    IN p_BookingID INT
)
BEGIN

    -- Cancel booking
    UPDATE Bookings
    SET IsActive = 0
    WHERE BookingID = p_BookingID;
END;
"""

# Create the updated AddBooking procedure
cursor.execute(cancel_booking_query)
connection.commit()

In [ ]:
# Call the CancelBooking stored procedure with specific values
cursor.callproc("CancelBooking", [7])

(7,)

In [ ]:
# Drop the existing GetMaxQuantity procedure if it exists
drop_procedure_query = "DROP PROCEDURE IF EXISTS GetMaxQuantity"
cursor.execute(drop_procedure_query)
connection.commit()

# Define the GetMaxQuantity stored procedure
get_max_quantity_query = """
CREATE PROCEDURE GetMaxQuantity(OUT max_quantity INT)
BEGIN
    DECLARE max_quantity INT;

    -- Get the maximum quantity from OrdersItem
    SELECT MAX(Quantity) INTO max_quantity FROM OrdersItem;

    -- Print or use the result as needed
    SELECT max_quantity AS MaxQuantity;
END;
"""

# Execute the procedure creation query
cursor.execute(get_max_quantity_query)
connection.commit()

# Call the stored procedure
cursor.callproc("GetMaxQuantity", [0])

# Fetch the result from the OUT parameter
max_quantity_result = cursor.stored_results()

# Iterate over the generator to fetch the result
for result in max_quantity_result:
    max_quantity = result.fetchone()

# Check if the result is not None before subscripting
if max_quantity is not None:
    print("Max Quantity:", max_quantity[0])
else:
    print("No result returned.")

Max Quantity: 3


In [ ]:
# Drop the existing ManageBooking procedure if it exists
drop_procedure_query = "DROP PROCEDURE IF EXISTS ManageBooking"
cursor.execute(drop_procedure_query)
connection.commit()

# Define the ManageBooking stored procedure
manage_booking_query = """
CREATE PROCEDURE ManageBooking()
BEGIN
    SELECT * FROM Bookings;
END
"""

# Execute the procedure creation query
cursor.execute(manage_booking_query)
connection.commit()



In [ ]:
# Call the stored procedure
cursor.callproc("little_lemon.ManageBooking")

# Fetch the result from the OUT parameter
manage_booking_result = cursor.stored_results()

# Print the manage_result
for row in manage_booking_result:
    print(row.fetchone())

(1, datetime.timedelta(seconds=68400), datetime.date(2025, 1, 1), 42, 'Batuhan', 'Baykal', 1, 3)


In [35]:
all_starters = """
        SELECT * FROM Starters;"""
cursor.execute(all_starters)

# Fetch all rows
result = cursor.fetchall()

# Print the result
for row in result:
    print(row)

(1, 'Olives')
(2, 'Flatbread')
(3, 'Minestrone')
(4, 'Tomato bread')
(5, 'Falafel')
(6, 'Hummus')


In [46]:
# Query that gives the number of bookings per cities
bookings_cities = """
        SELECT ct.City, COUNT(b.BookingID) AS total_booking_count
        FROM little_lemon.bookings AS b

        INNER JOIN little_lemon.cities AS ct
        ON ct.CityID=b.CityID

        GROUP BY ct.City
        ORDER BY total_booking_count DESC;"""
cursor.execute(bookings_cities)

# Fetch all rows
result = cursor.fetchall()

# Print the result
for row in result:
    print(row)

('Ongjin', 1)
('Quince Mil', 1)
('Susaki', 1)


In [47]:
# Query that gives the number of orders per city.
orders_cities = """
        SELECT ct.City, COUNT(o.OrderID) AS total_order_count
        FROM little_lemon.orders AS o

        INNER JOIN little_lemon.address AS a
        ON o.AddressID=a.AddressID

        INNER JOIN little_lemon.cities AS ct
        ON a.CityID=ct.CityID

        GROUP BY  ct.City
        ORDER BY total_order_count DESC;"""
cursor.execute(orders_cities)

# Fetch all rows
result = cursor.fetchall()

# Print the result
for row in result:
    print(row)

('Pindiga', 4)
('Cruz Alta', 3)
('Narathiwat', 2)
('Stockholm', 2)
('Petauke', 2)
('Hancheng', 2)
('Dongxin', 2)
('TÃ¢n PhÃº', 2)
('Su-ngai Kolok', 2)
('Bajos de Haina', 2)
('Shuiting', 2)
('Yuejin', 2)
('Boavista', 2)
('Gunziying', 2)
('Thap Than', 2)
('Destrnik', 2)
('Sacramento', 2)
('Kasangulu', 2)
('Trinity Ville', 2)
('Prokhorovka', 2)
('Arias', 2)
('Guanagazapa', 2)
('Toledo', 2)
('Longju', 2)
('Phoenix', 2)
('Yueyang', 2)
('Queenstown', 2)
('Vitim', 2)
('Hamilton', 2)
('Smokvica', 2)
('Melaka', 2)
('Guadalupe', 2)
('Sumberejo', 2)
('Hongtang', 2)
('Yangyong', 2)
('Rozhniv', 2)
('MontbÃ©liard', 2)
('Ja Ela', 2)
('San Francisco', 2)
('Malunda', 2)
('Mirabel', 2)
('Rennes', 2)
('Napoli', 2)
('Decatur', 2)
('Villa del Rosario', 2)
('Lazaro Cardenas', 2)
('Daruoyan', 1)
('Ongjin', 1)
('Quince Mil', 1)
('Susaki', 1)
('Tobruk', 1)
('Rafael Hernandez Ochoa', 1)
('BelozÃ«rskoye', 1)
('PÃ³voa do Valado', 1)
('Galâ€™bshtadt', 1)
('Fuyang', 1)
('Wakayama-shi', 1)
('Qingshan', 1)
('Pueblo V

In [48]:
# Query that gives the number of orders per country.
orders_countries = """
        SELECT co.Country,  COUNT(o.OrderID) AS total_order_count
        FROM little_lemon.orders AS o

        INNER JOIN little_lemon.address AS a
        ON o.AddressID=a.AddressID

        INNER JOIN little_lemon.cities AS ct
        ON ct.CityID=a.CityID

        INNER JOIN little_lemon.countries AS co
        ON ct.CountryID=co.CountryID

        GROUP BY co.Country
        ORDER BY total_order_count DESC;"""
cursor.execute(orders_countries)

# Fetch all rows
result = cursor.fetchall()

# Print the result
for row in result:
    print(row)



('China', 186)
('Indonesia', 108)
('Russia', 70)
('Philippines', 47)
('Brazil', 37)
('Portugal', 34)
('United States', 31)
('France', 30)
('Poland', 24)
('Japan', 21)
('Sweden', 19)
('Ukraine', 17)
('Thailand', 17)
('Argentina', 17)
('Colombia', 15)
('Peru', 14)
('Czech Republic', 14)
('Greece', 13)
('Nigeria', 12)
('Canada', 12)
('Mexico', 11)
('Albania', 8)
('Afghanistan', 7)
('Belarus', 6)
('Kazakhstan', 6)
('Croatia', 6)
('South Africa', 6)
('Armenia', 5)
('Vietnam', 5)
('Uganda', 5)
('Pakistan', 5)
('Guatemala', 5)
('Malaysia', 5)
('Cuba', 5)
('Germany', 4)
('Honduras', 4)
('Bangladesh', 4)
('Nepal', 4)
('Venezuela', 4)
('Finland', 4)
('Gambia', 4)
('Iran', 4)
('Ecuador', 4)
('Kenya', 4)
('Panama', 3)
('Azerbaijan', 3)
('Zambia', 3)
('Trinidad and Tobago', 3)
('Tunisia', 3)
('Micronesia', 3)
('Democratic Republic of the Congo', 3)
('Mongolia', 3)
('South Korea', 3)
('Moldova', 3)
('Madagascar', 3)
('Slovenia', 3)
('Zimbabwe', 3)
('Latvia', 3)
('Mali', 3)
('Morocco', 3)
('Netherlan

In [65]:
# Let's close the cursor and the connection
if connection.is_connected():
    connection.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

MySQL connection is closed.
